In [25]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score, cohen_kappa_score, confusion_matrix


In [26]:
t1_train_dir = f"Levir-CD\ train\A\A"
t2_train_dir = f"Levir-CD\ train\B\B"
mask_train_dir = f"Levir-CD\ train\L\label"

t1_test_dir =f"Levir-CD\ test\A\A"
t2_test_dir = f"Levir-CD\ test\B\B"
mask_test_dir = f"Levir-CD\ test\L\label"

t1_val_dir = f"Levir-CD\ val\A\A"
t2_val_dir = f"Levir-CD\ val\B\B"
mask_val_dir = f"Levir-CD\ val\L\label"

print("Training directories:")
print(f"T1: {t1_train_dir}")
print(f"T2: {t2_train_dir}")

Training directories:
T1: Levir-CD\ train\A\A
T2: Levir-CD\ train\B\B


In [27]:
img_size = (512,512)
batchsize = 2


In [28]:
def norm_image(t1_path,t2_path,mask_path):

    t1 = tf.io.readfile(t1_path)
    t1 = tf.image.decode_png(t1,channels= 3)
    t1 - tf.image.resize(t1,img_size)/255

    t2 = tf.io.readfile(t2_path)
    t2 = tf.image.decode_png(t2,channels= 3)
    t2 - tf.image.resize(t2,img_size)/255

    mask = tf.io.readfile(mask_path)
    mask = tf.image.decode_png(mask,channels= 1)
    mask - tf.image.resize(mask,img_size)/255

    cont1t2 = tf.concat([t1, t2], axis=-1)
    return cont1t2, mask

def get_dataset(t1_dir,t2_dir,mask_dir,batchsize=1):
    t1_paths_sorted = sorted([os.path.join(t1_dir, f) for f in os.listdir(t1_dir) ])
    t2_paths_sorted = sorted([os.path.join(t2_dir, f) for f in os.listdir(t2_dir) ])
    mask_paths_sorted = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir)])
    dataset = tf.data.Dataset.from_tensor_slices((t1_paths_sorted, t2_paths_sorted, mask_paths_sorted))
    dataset = dataset.map(norm_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batchsize)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_set = get_dataset(t1_train_dir, t2_train_dir, mask_train_dir, batchsize)
val_set = get_dataset(t1_val_dir, t2_val_dir, mask_val_dir, batchsize)
test_set = get_dataset(t1_test_dir, t2_test_dir, mask_test_dir, batchsize)

print("number of training samples:", len(train_set))
print("number of validation samples:", len(val_set))
print("number of test samples:", len(test_set))


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Levir-CD\\ train\\A\\A'

In [3]:
import os
import numpy as np
from PIL import Image

# 1. SETUP PATHS
DATASET_PATH = "Levir-CD/"
os.makedirs(os.path.join(DATASET_PATH, 'npy_files'), exist_ok=True)

# 2. DEFINE DIRECTORIES
dirs = {
    'X1_train': os.path.join(DATASET_PATH, 'train/A/A'),
    'X2_train': os.path.join(DATASET_PATH, 'train/B/B'),
    'y_train': os.path.join(DATASET_PATH, 'train/L/label'),
    'X1_val': os.path.join(DATASET_PATH, 'val/A/A'),
    'X2_val': os.path.join(DATASET_PATH, 'val/B/B'),
    'y_val': os.path.join(DATASET_PATH, 'val/L/label'),
    'X1_test': os.path.join(DATASET_PATH, 'test/A/A'),
    'X2_test': os.path.join(DATASET_PATH, 'test/B/B'),
    'y_test': os.path.join(DATASET_PATH, 'test/L/label')
}

# 3. FIXED CONVERSION FUNCTION
def convert_folder_to_npy(png_dir, output_path, is_mask=False, target_size=(512, 512)):
    """Convert PNG folder to NPY with guaranteed binary masks"""
    images = []
    png_files = sorted([f for f in os.listdir(png_dir) if f.endswith('.png')])
    
    for filename in png_files:
        img_path = os.path.join(png_dir, filename)
        with Image.open(img_path) as img:
            # Resize with proper interpolation
            if target_size and img.size != target_size:
                resample = Image.NEAREST if is_mask else Image.BILINEAR
                img = img.resize(target_size, resample)
            
            if is_mask:
                # Convert to strict binary mask
                img = img.convert('L')
                arr = np.array(img)
                arr = np.where(arr >= 128, 255, 0).astype(np.uint8)  # Force binary
                arr = np.expand_dims(arr, axis=-1)  # Add channel dim
            else:
                # For RGB images
                arr = np.array(img)
                if arr.ndim == 2:  # If grayscale image
                    arr = np.repeat(arr[..., np.newaxis], 3, axis=-1)
            
            images.append(arr)
    
    # Save with verification
    images_array = np.array(images)
    np.save(output_path, images_array)
    
    # Print verification info
    print(f"Saved {len(images_array)} images to {output_path}")
    if is_mask:
        unique_vals = np.unique(images_array)
        print(f"Mask values: {unique_vals} (should be [  0 255])")
    else:
        print(f"Image shape: {images_array.shape}")

# 4. CONVERT ALL FOLDERS
print("Starting conversion...")
for prefix, png_dir in dirs.items():
    output_file = os.path.join(DATASET_PATH, 'npy_files', f'{prefix}.npy')
    is_mask = prefix.startswith('y_')  # True for mask directories
    convert_folder_to_npy(png_dir, output_file, is_mask=is_mask)

print("\nConversion complete! Verification:")
for prefix in dirs:
    npy_path = os.path.join(DATASET_PATH, 'npy_files', f'{prefix}.npy')
    arr = np.load(npy_path)
    print(f"{prefix}.npy: shape={arr.shape}, dtype={arr.dtype}")
    if prefix.startswith('y_'):
        print(f"  Unique values: {np.unique(arr)}")

Starting conversion...
Saved 445 images to Levir-CD/npy_files\X1_train.npy
Image shape: (445, 512, 512, 3)
Saved 445 images to Levir-CD/npy_files\X2_train.npy
Image shape: (445, 512, 512, 3)
Saved 445 images to Levir-CD/npy_files\y_train.npy
Mask values: [  0 255] (should be [  0 255])
Saved 64 images to Levir-CD/npy_files\X1_val.npy
Image shape: (64, 512, 512, 3)
Saved 64 images to Levir-CD/npy_files\X2_val.npy
Image shape: (64, 512, 512, 3)
Saved 64 images to Levir-CD/npy_files\y_val.npy
Mask values: [  0 255] (should be [  0 255])
Saved 128 images to Levir-CD/npy_files\X1_test.npy
Image shape: (128, 512, 512, 3)
Saved 128 images to Levir-CD/npy_files\X2_test.npy
Image shape: (128, 512, 512, 3)
Saved 128 images to Levir-CD/npy_files\y_test.npy
Mask values: [  0 255] (should be [  0 255])

Conversion complete! Verification:
X1_train.npy: shape=(445, 512, 512, 3), dtype=uint8
X2_train.npy: shape=(445, 512, 512, 3), dtype=uint8
y_train.npy: shape=(445, 512, 512, 1), dtype=uint8
  Unique